In [108]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import urllib.parse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from tqdm import tqdm
import os

In [4]:
doubleou_ladder = 'https://pokemonshowdown.com/ladder/gen9doublesou'
ladder_plaintext = requests.get(doubleou_ladder).text

ladder_soup = BeautifulSoup(ladder_plaintext)

In [15]:
all_lines = ladder_soup.find('table').find_all('tr')[1:] # Remove first line, as they are indices for the table

In [35]:
dataframe = pd.DataFrame([{'rank': rank.text, 'user': username.text, 'user_href': username.find('a')['href'], 'elo': elo.text, 'gxe': gxe.text, 'rating': rating.text} for rank, username, elo, gxe, rating in [line.find_all('td') for line in all_lines]])

In [33]:
all_lines[0].find_all('td')[1].find('a')['href']

'/users/apricotpeach'

In [38]:
dataframe.to_csv('top_500_ladder.csv', index = False)

In [71]:
chrome_options = Options()
# chrome_options.add_argument('--headless')  # This line makes the browser headless
chrome_options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36")
driver = webdriver.Chrome(options=chrome_options)
    

In [104]:

def get_replays_from_user(username, replays_from_users_fstring = 'https://replay.pokemonshowdown.com/?user={user}&page={page}', startswith_condition_string = 'gen9', currentpage=1, driver=driver):
    encoded_user = urllib.parse.quote(username)
    replaylinks = []
    
    while True:

        url = replays_from_users_fstring.format(user = encoded_user, page = currentpage)
        _ = driver.get(url)

        blocklink_elements = driver.find_elements(By.CLASS_NAME, "blocklink")

        if not len(blocklink_elements): return replaylinks

        # Extract href attributes from the elements
        for element in blocklink_elements:
            href = element.get_attribute('href')

            # if startswith_condition_string in href.split('/')[-1]:
            replaylinks.append(href.split('?p2')[0] + '.json')

        currentpage += 1

In [ ]:
basepath = 'gameslogs'
os.makedirs(basepath)
for user in tqdm(dataframe['user'], desc='scrapping users...'):
    jsons = get_replays_from_user(user)
    for json_link in jsons:
        response = requests.get(json_link)
        json_data = response.json()
        with open(os.path.join(basepath, json_link.split('/')[-1]), 'w') as file:
            file.write(response.text)

        

scrapping users...:  22%|████▎              | 112/500 [17:55<2:22:06, 21.98s/it]